THIS WAS RUN ON GG COLAB

# Check

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
del drive

In [ ]:
!cat /proc/meminfo

In [ ]:
!cat /proc/cpuinfo

# download  + check packages

In [ ]:
!pip install sentinelhub

from sentinelhub import SHConfig

config = SHConfig()
config
del config

# code for download

In [ ]:
from sentinelhub import SHConfig
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

max_download_attempts = 3

config = SHConfig(instance_id='7e956740-b2bf-44ea-86ea-6b43c3c8ceda', sh_client_id='60e4df50-fb08-448c-b20d-8f8f97582d01', sh_client_secret = '3hWfbBQp^rXf(:k|#(au<^rplaBFo5_17/D]S#5}' )

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")


sb_coords_wgs84 = (16.300278,48.178735,16.320062,48.186948) #  (longitude and latitude coordinates of lower left and upper right corners)
resolution = 10
sb_bbox = BBox(bbox=sb_coords_wgs84, crs=CRS.WGS84)
sb_size = bbox_to_dimensions(sb_bbox, resolution=resolution) #

for collection in DataCollection.get_available_collections():
    print(collection)

In [ ]:
evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                units: "DN"
            }],
            output: {
                bands: 12,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12];
    }
"""

request_all_bands = SentinelHubRequest(
    evalscript=evalscript_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C, # SENTINEL2_L2A
            time_interval=("2023-09-26", "2023-09-27"),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=sb_bbox,
    size=sb_size,
    config=config,
)

all_bands_response = request_all_bands.get_data()

img_mean      = [1111.81236406, 824.63171476, 663.41636217, 445.17289745,
                          645.8582926, 1547.73508126, 1960.44401001, 1941.32229668,
                          674.07572865, 9.04787384, 1113.98338755, 519.90397929]   # means of 14 bands
img_mean = np.expand_dims(np.array(img_mean), axis=(0, 1))

imgs = np.array(all_bands_response)
bgr = imgs[0]/img_mean
bgr = bgr[:,:,1:4]

In [ ]:
def changeBGR2RGB(bgr_img):
  """
    convert BGR to RGB image using Min-Max scaling
  """
  if not isinstance(bgr_img, np.ndarray):
    bgr_img = bgr_img.numpy()

  red   = (bgr_img[:,:,2]-np.min(bgr_img[:,:,2])) / (np.max(bgr_img[:,:,2])-np.min(bgr_img[:,:,2]))
  green = (bgr_img[:,:,1]-np.min(bgr_img[:,:,1])) / (np.max(bgr_img[:,:,1])-np.min(bgr_img[:,:,1]))
  blue  = (bgr_img[:,:,0]-np.min(bgr_img[:,:,0])) / (np.max(bgr_img[:,:,0])-np.min(bgr_img[:,:,0]))

  red   = np.expand_dims(red, axis=2)
  green = np.expand_dims(green, axis=2)
  blue  = np.expand_dims(blue, axis=2)

  rgb   = np.concatenate((red, green), axis=-1)
  rgb   = np.concatenate((rgb, blue), axis=-1)

  return rgb

rgb = changeBGR2RGB(bgr)
print('\n =================================== ')
print(np.min(rgb[:,:,0]), np.max(rgb[:,:,0]))
print(np.min(rgb[:,:,1]), np.max(rgb[:,:,1]))
print(np.min(rgb[:,:,2]), np.max(rgb[:,:,2]))
print('\n =================================== ')

plt.imshow(rgb)
plt.show()